In [10]:
%load_ext autoreload
%autoreload 2

ModuleNotFoundError: No module named 'IPython.extensions.deduperreload'

In [11]:
import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))

In [12]:
import pandas as pd
from src.config import TRANSFORMED_DATA_DIR

df = pd.read_parquet(TRANSFORMED_DATA_DIR / "tabular_data.parquet")

In [13]:
from datetime import datetime

from src.data_utils import split_time_series_data

X_train, y_train, X_test, y_test = split_time_series_data(
    df,
    cutoff_date=datetime(2023, 9, 1, 0, 0, 0),
    target_column="target"
)

print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(55900, 674)
(55900,)
(31720, 674)
(31720,)


In [14]:
past_ride_columns = [c for c in X_train.columns if c.startswith("rides_")]
X_train_only_numeric = X_train[past_ride_columns]
X_test_only_numeric = X_test[past_ride_columns]

In [15]:
import lightgbm as lgb
model = lgb.LGBMRegressor()
model.fit(X_train_only_numeric, y_train)

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.052562 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 159651
[LightGBM] [Info] Number of data points in the train set: 55900, number of used features: 672
[LightGBM] [Info] Start training from score 11.110286


LGBMRegressor()

In [16]:
from sklearn.metrics import mean_absolute_error
predictions = model.predict(X_test_only_numeric)

In [17]:
test_mae = mean_absolute_error(y_test, predictions)
print(f"{test_mae:.4f}")

3.3054


In [18]:
from src.experiment_utils import set_mlflow_tracking, log_model_to_mlflow
from dotenv import load_dotenv
import os
load_dotenv() 

mlflow = set_mlflow_tracking()
log_model_to_mlflow(model, X_test_only_numeric, "LGBMRegressor", "mean_absolute_error", score=test_mae)

INFO:src.experiment_utils:MLflow tracking URI and credentials set.
2025/03/02 21:54:47 INFO mlflow.tracking.fluent: Experiment with name 'LGBMRegressor' does not exist. Creating a new experiment.
INFO:src.experiment_utils:Experiment set to: LGBMRegressor
INFO:src.experiment_utils:Logged mean_absolute_error: 3.3054128762542527
/opt/anaconda3/envs/con_env1/lib/python3.11/site-packages/mlflow/types/utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/do

🏃 View run polite-dolphin-552 at: https://dagshub.com/nelavalli.sandeep/Nyc_taxi_project1.mlflow/#/experiments/3/runs/695c413d633648e890db7c238d569ee8
🧪 View experiment at: https://dagshub.com/nelavalli.sandeep/Nyc_taxi_project1.mlflow/#/experiments/3
